In [2]:
#EXECUTION
from CircleNet.execution import get_parameters,Write_csv, save_agents
from CircleNet.shanghai_platform_extension import *
from CircleNet.system import get_id


import config as parameter_file#parameter
d=Simu_counter()#displayer
d.start()

name="data/{}_{}.csv".format(get_id(),"general")

result_file=Write_csv(name,general_data_to_save)#results

for PARAMETERS,message in get_parameters(parameter_file):
    d.new_sim(message)#update message in displayer
    simu=create_simulation(**PARAMETERS)#creation
    simu.set_action(lambda x: d.change(x.time))
    simu()#execution
    results=extract_result_data(simu)#extraction
    result_file.write(**results)#save
    save_agents(simu,*particular_files)
    
d.quit()

In [ ]:
#DRAW ANIMATION
dr=Positions_drawing(simu)
dr.set_options(margin=20,real_size=(10,10),time_interval=30,time_coef=1000,repetition=True,hide_axes=True)
dr()

In [ ]:
#PLOT ROUTES
plot_routes(simu)
plot_relative_routes(simu)

In [ ]:
#TODO

#test ploting hist etc...
#problem of results?
#change name to value of time
#finish beautiful file

In [3]:
a="akjhfjkhz"

In [9]:
if a :
    print(a)
    

NameError: name 'a' is not defined

In [8]:
del a